# Imports

In [1]:
#!pip install openai
#!pip install PyPDF2
#!pip install openpyxl
#!pip install unidecode

import pandas as pd
from openai import OpenAI
from confighowdy import OPENAI_API_KEY, CV_FOLDER, FILE_PARTNER_DATABASE, PROCESSED_FILE

#from prompt_handler import *

In [2]:
%run functions-howdy.ipynb


# Client instance:

In [3]:

# Initialize client
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

# Partner CV processing

In [4]:
# Path to the folder containing the PDF files
cvs_dict = process_all_pdfs(CV_FOLDER)

In [5]:
# CV sanity check

# Get the keys of the dictionary
cvs_dict_keys = list(cvs_dict.keys())

# Print processed CV sample
print(cvs_dict[cvs_dict_keys[1]])

Sofía Vargas
I am dedicated to developing innovative software solutions for the sports industry. My focus is on
creating applications that enhance athletic performance and engage sports enthusiasts.
Name: Sofía Vargas
Company: PeakPerformance Sports
Technologies: Vue.js, JavaScript, Cassandra
Education
- BSc Sports Management - Universidad de Madrid
Experience
- Software Developer at PeakPerformance Sports (3 years)
- Intern at SportsTech (1 year)
Additional Skills
- Sports Technology
- Web Development
- User Experience


# Get partner information database

In [6]:
partner_information_dataframe = get_partner_information(FILE_PARTNER_DATABASE)

In [7]:
# Partner info sanity check
partner_information_dataframe.sample(5)

,Name,Partner,Technologies,Industry
66,Hugo Soto,UrbanNest Realty,"Vue.js, Kotlin, PostgreSQL",Real Estate
59,Ignacio Paredes,Oceanic Foods,"Angular, Node.js, SQLite",Food Industry
62,Manuela Diaz,PulseTech Systems,"Angular, Flask, PostgreSQL",Technology
39,Sofia Molina,Horizon BioSciences,"Vue.js, Java, SQL Server",Biotechnology
6,Ana Gomez,QuantumMed Innovations,"React, Python, SQLite",Healthcare


# Process partner's cvs

In [8]:
process_partners_cvs(cvs_dict)

{'Francisco_Moreno_Resume.pdf': '1. Aerospace Software Development\n2. Performance Optimization\n3. Application Reliability\n4. Engineering Solutions\n5. Aerospace Technology Advancements',
 'Sofia_Vargas_Resume.pdf': '1. Sports Technology\n2. Web Development\n3. User Experience\n4. Athletic Performance Enhancement\n5. Vue.js and JavaScript Development',
 'Alberto_Quintero_Resume.pdf': '1. Cloud Computing\n2. Data Security\n3. Network Infrastructure\n4. Software Development (Angular, Java)\n5. Database Management (Cassandra)',
 'Nicolas_Gutierrez_Resume.pdf': '1. Renewable Energy Software Solutions\n2. Front-end Development using React and JavaScript\n3. Database Management with MongoDB\n4. Environmental Engineering\n5. Sustainable Technology Innovation',
 'Catalina_Cruz_Resume.pdf': '1. Financial Analysis\n2. Investment Strategies\n3. Software Development for FinTech\n4. Data Analysis\n5. Financial Modeling',
 'Sofia_Rodriguez_Resume.pdf': '1. Front-end Development (Angular)  \n2. Bac

In [9]:
df = pd.read_csv(PROCESSED_FILE)
df_cvs_processed = organize_dataframe(df)

In [10]:
# Sanity check
df_cvs_processed.head()

,Name,Processed_CV,Processed_CV_list
0,Francisco Moreno,1. Aerospace Software Development\n2. Performa...,"[Aerospace Software Development, Performance O..."
1,Sofia Vargas,1. Sports Technology\n2. Web Development\n3. U...,"[Sports Technology, Web Development, User Expe..."
2,Alberto Quintero,1. Cloud Computing\n2. Data Security\n3. Netwo...,"[Cloud Computing, Data Security, Network Infra..."
3,Nicolas Gutierrez,1. Renewable Energy Software Solutions\n2. Fro...,"[Renewable Energy Software Solutions, Front-en..."
4,Catalina Cruz,1. Financial Analysis\n2. Investment Strategie...,"[Financial Analysis, Investment Strategies, So..."


# Create full profiles

In [11]:
# Realizar el merge basado en la columna 'Name'
profile_df = pd.merge(partner_information_dataframe, df_cvs_processed, on="Name", how="inner")


In [12]:
profile_df.to_csv("experts_profile.csv")

# Embeddings from profiles

In [13]:
normalized_embeddings = process_and_normalize_embeddings(client, profile_df)

The embedding matrix contains: 96 samples
Each embedding has: 1536 features


In [14]:
print("Embedding examples:")
print(normalized_embeddings[0])  # First embedding
print(normalized_embeddings[1])  # Second embedding
print(normalized_embeddings[-1])  # Last embedding


Embedding examples:
[ 0.02341388 -0.00624505 -0.01109256 ... -0.01348256  0.00174524
 -0.00140429]
[ 0.02447718 -0.00612943 -0.01300899 ... -0.01393483  0.00265417
 -0.00081222]
[-0.00267151 -0.02462849 -0.00130753 ... -0.03272853 -0.00721067
  0.0109802 ]


# User GPT request

In [54]:
%run functions-howdy.ipynb

# Initialize client
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [55]:
user_prompt = """
    Im working for a healthcare system company and there are a lot of concepts that I don't understand.
    Can you refer me to someone who knows about the topic?
"""
expert_search_response = generate_response(user_prompt)

print(expert_search_response)

Wait a few moments while I search the database
This shouldn't take more than 20 seconds
Certainly! From the list of experts, I found a few who can be of great help.

1. Jose Munoz: Jose is an experienced professional working at QuantumMed Innovations, a healthcare company. He has a solid background in Health Tech and Medical Software. His skills in Backend Development and Full-Stack Development using React, Spring Boot, and PostgreSQL can certainly help you grasp the complex concepts you are encountering.

2. Isabel Ramirez: Also working at QuantumMed Innovations, Isabel has a strong expertise in Biomedical Engineering, Healthcare Technology Development, and Medical Software Development where she utilizes tools like Vue.js, Java, and PostgreSQL. Isabel's knowledge extends to Front-End Development and Database Management, which could also be useful to your learning journey.

3. Fernanda Ruiz: Also at QuantumMed Innovations, Fernanda specializes in Biotechnology and Healthcare Technology

In [56]:
user_prompt = """ Hi chat gpt"""

simple_response = generate_response(user_prompt)

print(simple_response)

Wait a few moments while I search the database
This shouldn't take more than 20 seconds
Hello! How can I assist you today? Whether you need recommendations, guidance, or just someone to answer your questions, I'm here to help.


In [57]:
user_prompt = "Im trying to find some painter to give me advice"

failed_expert_search_response = generate_response(user_prompt)

print(failed_expert_search_response)

Wait a few moments while I search the database
This shouldn't take more than 20 seconds
I'm sorry, but among the available experts we have right now, none of them specialize in painting or similar areas. 

Santiago Martinez who works with TechNova Solutions has expertise in Frontend Development, User Interface Design, JavaScript Frameworks, Team Collaboration, and Database Management (PostgreSQL). He is very adept with Vue.js, JavaScript, PostgreSQL.

Another expert associated with us is Javier Ramirez from BlueWave Logistics. He can assist with Backend Development, Java Programming, MongoDB Database Management, Problem Solving and Critical Thinking and Software Engineering and Adaptability, and he is experienced with Angular, Java, MongoDB technologies.

There is also Martin Herrera, who is also part of TechNova Solutions. His skills include Web Technologies, Scalable Application Development, Frontend Development with React, Backend Development with Java, and Database Management with 